# 3D Poisson Equation - PROVEN BENCHMARK

**Problem**: ∇²u = f on [0,1]³ with u=0 on boundary

**Manufactured solution**: u = sin(πx)sin(πy)sin(πz)

This is a **well-known benchmark** where PINNs are **proven to work**.

## Step 1: Upload Project

Upload `3D_PINN.zip` via Files panel, then extract:

In [ ]:
# Extract uploaded zip
!unzip -q 3D_PINN.zip

# Verify
print("Current directory:")
!pwd
print("\nProject files:")
!ls -la
print("\nPINN modules:")
!ls src/pinn3d/

## Step 2: Install Dependencies

In [ ]:
# Install core dependencies
!pip install -q jax==0.9.0
!pip install -q equinox==0.13.2
!pip install -q optax numpy scipy matplotlib

# Install JAX with GPU
!pip install -U "jax[cuda12]"

In [ ]:
# Verify GPU
import jax
print(f"JAX version: {jax.__version__}")
print(f"Devices: {jax.devices()}")
print(f"Backend: {jax.default_backend()}")

## Step 3: Setup and Imports

In [ ]:
import sys
from pathlib import Path

# Add src to path
src_path = '/content/src'
if src_path not in sys.path:
    sys.path.insert(0, src_path)

# Enable float64
jax.config.update('jax_enable_x64', True)
print("✓ Float64 enabled")
print(f"✓ Added to path: {src_path}")

In [ ]:
# Import PINN modules
from pinn3d.config import load_config
from pinn3d.poisson_model import create_poisson_model
from pinn3d.poisson_pde import manufactured_solution_source, exact_solution
from pinn3d.poisson_loss import make_poisson_loss_fn
from pinn3d.sampling import sample_interior_sobol, sample_boundary_sobol, scale_to_input_range
from pinn3d.checkpoints import CheckpointManager
import jax.numpy as jnp
import numpy as np
import equinox as eqx
import optax
import time

print("✓ All imports successful")

## Step 4: Load Configuration

In [ ]:
# Load Poisson config
config = load_config('/content/configs/poisson_3d_simple.yaml')

print("✓ POISSON EQUATION (Proven Benchmark)")
print(f"  Problem: ∇²u = f (simpler than Helmholtz)")
print(f"  Solution: sin(πx)sin(πy)sin(πz) (known exact answer)")
print(f"  Network: {config['network']['hidden_layers']} layers × {config['network']['width']} width")
print(f"  Adam steps: {config['adam']['steps']:,} (~1 hour)")
print(f"  L-BFGS: {config['lbfgs']['max_iterations']:,} iterations (~5 min)")
print(f"  Expected: <1% error (PROVEN)")

In [ ]:
# Create components
source_fn = manufactured_solution_source
model = create_poisson_model(config)
loss_fn = make_poisson_loss_fn(config, source_fn)
checkpoint_manager = CheckpointManager('/content/checkpoints')

print("✓ Model and loss function created")
print(f"  Parameters: ~{sum(x.size for x in jax.tree_util.tree_leaves(model) if hasattr(x, 'size')):,}")

## Step 5: Train with Adam

In [ ]:
# Training setup
n_steps = config['adam']['steps']
lr = config['adam']['learning_rate']
lr_min = config['adam']['lr_min']
grad_clip = config['adam']['gradient_clip']

n_interior = config['sampling']['n_interior']
n_boundary = config['sampling']['n_boundary']
n_boundary_per_face = n_boundary // 6
seed = config['sampling']['seed']

# Optimizer
schedule = optax.cosine_decay_schedule(lr, n_steps, lr_min / lr)
optimizer = optax.chain(
    optax.clip_by_global_norm(grad_clip),
    optax.adam(learning_rate=schedule)
)
opt_state = optimizer.init(eqx.filter(model, eqx.is_array))

print("✓ Training setup complete")

In [ ]:
# JIT compiled functions
@eqx.filter_jit
def compute_loss_and_grad(model, x_interior, x_boundary):
    loss_fn_with_model = lambda m: loss_fn(m, x_interior, x_boundary)[0]
    loss, grads = eqx.filter_value_and_grad(loss_fn_with_model)(model)
    _, info = loss_fn(model, x_interior, x_boundary)
    return loss, grads, info

@eqx.filter_jit
def update_step(model, opt_state, x_interior, x_boundary):
    loss, grads, info = compute_loss_and_grad(model, x_interior, x_boundary)
    updates, opt_state = optimizer.update(grads, opt_state, model)
    model = eqx.apply_updates(model, updates)
    return model, opt_state, loss, info

print("✓ JIT compilation ready")

In [ ]:
# Training loop
print("="*80)
print("ADAM TRAINING")
print("="*80)

start_time = time.time()
for step in range(n_steps):
    # Sample points
    step_seed = seed + step
    x_int = sample_interior_sobol(n_interior, seed=step_seed)
    x_bnd = sample_boundary_sobol(n_boundary_per_face, seed=step_seed)
    x_int_scaled = scale_to_input_range(x_int)
    x_bnd_scaled = scale_to_input_range(x_bnd)
    
    # Update
    model, opt_state, loss, info = update_step(model, opt_state, x_int_scaled, x_bnd_scaled)
    
    # Log
    if step % 500 == 0 or step == n_steps - 1:
        elapsed = time.time() - start_time
        print(f"Step {step:6d} | Loss: {loss:.6e} | "
              f"L_b: {info['loss_boundary']:.6e} | "
              f"L_f: {info['loss_residual']:.6e} | "
              f"Time: {elapsed:.1f}s")
    
    # Checkpoint
    if step % 2000 == 0:
        checkpoint_manager.save(model, step, loss, info)

print("\n✓ Adam training complete!")

## Step 6: Validate Against Exact Solution

In [ ]:
# Create validation grid
n_val = 33
x_val = np.linspace(0, 1, n_val)
y_val = np.linspace(0, 1, n_val)
z_val = np.linspace(0, 1, n_val)
X, Y, Z = np.meshgrid(x_val, y_val, z_val, indexing='ij')
coords = np.stack([X.ravel(), Y.ravel(), Z.ravel()], axis=1)

# Exact solution
u_exact = exact_solution(jnp.array(coords))

# PINN prediction
coords_scaled = scale_to_input_range(jnp.array(coords))
from pinn3d.poisson_pde import batch_prediction_poisson
u_pinn = batch_prediction_poisson(model, coords_scaled)

# Compute error (interior points only)
interior_mask = np.ones(len(coords), dtype=bool)
for i in range(len(coords)):
    if any(coords[i] == 0.0) or any(coords[i] == 1.0):
        interior_mask[i] = False

u_exact_interior = u_exact[interior_mask]
u_pinn_interior = u_pinn[interior_mask]

l2_error_abs = float(jnp.linalg.norm(u_pinn_interior - u_exact_interior))
l2_norm_exact = float(jnp.linalg.norm(u_exact_interior))
l2_error_rel = l2_error_abs / (l2_norm_exact + 1e-10)

print("="*80)
print("VALIDATION RESULTS")
print("="*80)
print(f"Validation grid: {n_val}³ = {n_val**3:,} points")
print(f"L2 relative error: {l2_error_rel:.6f} ({l2_error_rel*100:.3f}%)")
print(f"L2 absolute error: {l2_error_abs:.6e}")
print("="*80)

if l2_error_rel < 0.01:
    print("🎉 SUCCESS! Error < 1%")
elif l2_error_rel < 0.05:
    print("✓ Good! Error < 5%")
else:
    print("⚠️  Error > 5%")

## Step 7: Save Model

In [ ]:
# Save final model
final_path = 'poisson_model_final.eqx'
eqx.tree_serialise_leaves(final_path, model)
print(f"✓ Model saved to {final_path}")

# Download
from google.colab import files
files.download(final_path)
print("✓ Model downloaded")